<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_termin11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 11

Vežba obuhvata zadatke:
*   Zadatak 1 - Konvolucija matrice



## Preduslovi
Pre nego započnete implementaciju zadataka, **obavezno** izvršite kod iz ćelije ispod. Kod će:
- skinuti kompresovanu biblioteku `libwb` koja se koristi u zadacima i raspakovati je na putanju /usr/local/libwb.
- skinuti set podataka za testiranje rešenja zadatka i raspakovati ga na putanju /home/cuda

In [ ]:
%%bash

######################### PREUZMI I PODESI libwb ##############################
# preuzmi libwb.zip ako već nije preuzet
[[ -f libwb.zip ]] || wget -O libwb.zip "https://drive.google.com/u/0/uc?id=1-j31qXJJOBFKuNc4qzTjTU-Gji8HARh0&export=download"

# otpakuj
unzip -oqqd /usr/local libwb.zip


################### PREUZMI I PODESI set za testiranje ########################
# preuzmi dataset.zip ako vec nije preuzet
[[ -f dataset.zip ]] || wget -O dataset.zip "https://drive.google.com/u/0/uc?id=1wMYQvl2oadRCT0SaBQY4sZwM8QQaWns_&export=download"

# otpakuj u direktorijum u kojem ce se nalaziti izvrsna datoteka resenja
mkdir -p /home/cuda/
unzip -oqqd /home/cuda/ dataset.zip

Po završetku prethodne ćelije, možete pokrenuti narednu koja proverava da li `libwb` i `dataset` direktorijumi postoje na predviđenim putanjama. Ukoliko je sve u redu, obe naredbe će ispisati absolutne putanje do direktorijuma. U suprotnom će prijaviti da jedan ili oba direktorijuma nisu pronađena. U tom slučaju treba da se vratite na log prethodne ćelije i potražite gde je došlo do greške.

In [ ]:
%%bash

ls -d /usr/local/libwb/
ls -d /home/cuda/

## [Opciono] Deinstalacija preduslova
Izvršavanjem ćelije ispod brišete instaliranu biblioteku `libwb` i set podataka. Izvršite ovu ćeliju samo ako želite da resetujete instalaciju kako biste reinstalirali preduslove.

In [ ]:
%%bash

# obrisi libwb biblioteku
[[ -d /usr/local/libwb ]] && rm -r /usr/local/libwb/ && echo "Obrisana libwb biblioteka."
[[ -f libwb.zip ]] && rm libwb.zip && echo "Obrisana libwb.zip arhiva."

# obrisi set podataka za testiranje
[[ -d /home/cuda/dataset ]] && rm -r /home/cuda/dataset && echo "Obrisan set podataka."
[[ -f dataset.zip ]] && rm dataset.zip && echo "Obrisana dataset.zip arhiva."

## Zadatak 1 - Konvolucija matrica

Implementirati osnovni algoritam za konvoluciju ulazne RGB slike. Ulazna
slika je dimenzija $width \times height \times numchannels$, gde je broj
kanala fiksno 3 i odgovara R, G i B kanalima. Drugim rečima, svaki
piksel slike je predstavljen sa tri vrednosti koje predstavljaju crvenu,
zelenu i plavu komponentu boje posmatranog piksela. Slika je
linearizovana vrstu po vrstu, a vrednosti kanala piksela su u memoriji
zadate kao uzastopne vrednosti.\
Konvolucija RGB slike se svodi na računanje nove vrednosti svih kanala
svih piksela na slici. To znači da je u indeksiranje ulazne slike
potrebno uračunati i informaciju o kanalima, pa se određenom kanalu za
piksel u vrsti `yIndex` i koloni `xIndex` može pristupiti kao:

``` {.c}
index = (yIndex * width + xIndex) * channels + channelIndex
```

Za crvenu komponentu piksela važi da je `channelIndex = 0`, za zelenu
`channelIndex = 1` i za plavu `channelIndex = 2`.\
Konvoluciona maska je fiksno veličine $5\times5$. Radi efikasnijeg
iskorišćenja memorije, konvolucionu masku treba sačuvati u konstantnoj
memoriji koja omogućava keširanje vrednosti maske. Ovo se može postići
definisanjem parametra kernela koji odgovara konvolucionoj masci kao
`const float * __restrict__ M`, ako pretpostavimo da je `M` naziv
pokazivača preko kojeg se pristupa elementima konvolucione maske.\
Opšta formula za konvoluciju slike sa konvolucionim kernelom `M`
dimenzija $5\times5$, gde je sa `I` obeležena ulazna slika, a sa `P`
izlazna konvolvirana slika je:

$$P_{i,j,c} = \sum_{x=-2}^{2} \sum_{y=-2}^{2} I_{i+x,j+y,c} * M_{x,y}$$

U formuli, $P_{i,j,c}$ je kanal $c$ piksela izlazne slike na poziciji
$i,j$, $I_{i,j,c}$ je kanal $c$ piksela ulazne slike na poziciji $i,j$ i
$M_{x,y}$ je vrednost maske na poziciji $x,y$. Treba obezbediti da
vrednost kanala izlaznih piksela bude u intervalu \[0,1\].

Instrukcije
===========

Dopuniti zadatu postavku zadatka implementacijom nedostajućih delova
koda. Implementirati:

-   alokaciju memorije na uređaju,
-   kopiranje podataka iz memorije domaćina u memoriju uređaja,
-   inicijalizaciju dimenzija mreže i bloka,
-   poziv kernela,
-   kopiranje podataka sa memorije uređaja u memoriju domaćina i
-   dealokaciju memorije uređaja,
-   konvolucioni kernel

### Pseudo-kod rešenja

``` {.pascal linenos=""}
maskWidth := 5
maskRadius := maskWidth/2   // this is integer division, so the result is 2
for i from 0 to height do
    for j from 0 to width do
        for k from 0 to channels
            accum := 0
            for y from -maskRadius to maskRadius do
                for x from -maskRadius to maskRadius do
                    xOffset := j + x
                    yOffset := i + y
                    if xOffset >= 0 and xOffset < width and
                       yOffset >= 0 and yOffset < height then
                        imagePixel := I[(yOffset * width + xOffset) * channels + k]
                        maskValue := K[(y+maskRadius)*maskWidth+x+maskRadius]
                        accum += imagePixel * maskValue
                    end
                end
            end
            // pixels are in the range of 0 to 1
            P[(i * width + j)*channels + k] = clamp(accum, 0, 1)
        end
    end
end

// where clamp is defined as
def clamp(x, lower, upper)
    return min(max(x, lower), upper)
end
```

In [ ]:
%%writefile /home/cuda/matrix_convolution.cu

#include <wb.h>

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      return -1;                                                          \
    }                                                                     \
  } while (0)

#define Mask_width 5
#define Mask_radius Mask_width / 2
#define clamp(x) (min(max((x), 0.0), 1.0))

//@@ INSERT CODE HERE

int main(int argc, char *argv[]) {
  wbArg_t arg;
  int maskRows;
  int maskColumns;
  int imageChannels;
  int imageWidth;
  int imageHeight;
  char *inputImageFile;
  char *inputMaskFile;
  wbImage_t inputImage;
  wbImage_t outputImage;
  float *hostInputImageData;
  float *hostOutputImageData;
  float *hostMaskData;
  float *deviceInputImageData;
  float *deviceOutputImageData;
  float *deviceMaskData;

  arg = wbArg_read(argc, argv); /* parse the input arguments */

  inputImageFile = wbArg_getInputFile(arg, 0);
  inputMaskFile  = wbArg_getInputFile(arg, 1);

  inputImage   = wbImport(inputImageFile);
  hostMaskData = (float *)wbImport(inputMaskFile, &maskRows, &maskColumns);

  assert(maskRows == 5);    /* mask height is fixed to 5 in this mp */
  assert(maskColumns == 5); /* mask width is fixed to 5 in this mp */

  imageWidth    = wbImage_getWidth(inputImage);
  imageHeight   = wbImage_getHeight(inputImage);
  imageChannels = wbImage_getChannels(inputImage);

  outputImage = wbImage_new(imageWidth, imageHeight, imageChannels);

  hostInputImageData  = wbImage_getData(inputImage);
  hostOutputImageData = wbImage_getData(outputImage);

  wbTime_start(GPU, "Doing GPU Computation (memory + compute)");

  wbTime_start(GPU, "Doing GPU memory allocation");
  //@@ INSERT CODE HERE
  wbTime_stop(GPU, "Doing GPU memory allocation");

  wbTime_start(Copy, "Copying data to the GPU");
  //@@ INSERT CODE HERE
  wbTime_stop(Copy, "Copying data to the GPU");

  wbTime_start(Compute, "Doing the computation on the GPU");
  //@@ INSERT CODE HERE

//  convolution<<<dimGrid, dimBlock>>>(deviceInputImageData, deviceMaskData,
//                                     deviceOutputImageData, imageChannels,
//                                     imageWidth, imageHeight);
  wbTime_stop(Compute, "Doing the computation on the GPU");

  wbTime_start(Copy, "Copying data from the GPU");
  //@@ INSERT CODE HERE
  cudaMemcpy(hostOutputImageData, deviceOutputImageData,
             imageWidth * imageHeight * imageChannels * sizeof(float),
             cudaMemcpyDeviceToHost);
  wbTime_stop(Copy, "Copying data from the GPU");

  wbTime_stop(GPU, "Doing GPU Computation (memory + compute)");

  wbSolution(arg, outputImage);

  //@@ Insert code here

  free(hostMaskData);
  wbImage_delete(outputImage);
  wbImage_delete(inputImage);

  return 0;
}

### Kompajliranje izvornog koda
Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/cuda
nvcc matrix_convolution.cu -o matrix_convolution \
  -I /usr/local/libwb/ \
  -L /usr/local/libwb/lib \
  -lwb -lcuda

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/cuda/matrix_convolution ]] && echo "Postoji." || echo "Ne postoji." 

### Pokretanje rešenja nad jednim primerom

In [ ]:
%%bash

cd /home/cuda
LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./matrix_convolution \
      -i dataset/0/input0.ppm,dataset/0/input1.raw \
      -o output.ppm -e dataset/0/output.ppm \
      -t image

### Pokretanje rešenja nad svim primerima

Nakon uspešnog testiranja rešenja na jednom primeru, možete pokrenuti i testiranje rešenja na svim primerima iz seta podataka.

In [ ]:
%%bash

cd /home/cuda

n=$(ls -dl dataset/* | wc -l)

for (( i=0; i<$n; i=i+1 ))
do
  echo "Testiram primer $i."
  echo "-------------------"
  LD_LIBRARY_PATH="/usr/local/libwb/lib:$LD_LIBRARY_PATH" ./matrix_convolution \
      -i dataset/0/input0.ppm,dataset/0/input1.raw \
      -o output.ppm -e dataset/0/output.ppm \
      -t image
  echo ""
done